In [1]:
!pip install pdfplumber

     ---------------------------------------- 0.0/40.1 kB ? eta -:--:--
     -------------------------------------- 40.1/40.1 kB 961.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.5 kB ? eta -:--:--
     -------- ------------------------------- 10.2/48.5 kB ? eta -:--:--
     -------------------------------------- 48.5/48.5 kB 490.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/58.0 kB ? eta -:--:--
   ---------------------------------------- 58.0/58.0 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB 991.0 kB/s eta 0:00:06
    --------------------------------------- 0.1/5.6 MB 660.6 kB/s eta 0:00:09
   - -------------------------------------- 0.1/5.6 MB 950.9 kB/s eta 0:00:06
   - -------------------------------------- 0.2/5.6 MB 985.7 kB/s eta 0:00:06
   - --------------------


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pdfplumber

# Open the PDF file
with pdfplumber.open('./data/Team_Geek_Book.pdf') as pdf:
    # Extract text from each page
    text = ''
    for page in pdf.pages:
        text += page.extract_text() + '\n'

# Save extracted text to a file with UTF-8 encoding
with open('team_geek_text.txt', 'w', encoding='utf-8') as f:
    f.write(text)

print('Text extracted and saved to team_geek_text.txt')

In [41]:
!pip install sentencepiece


   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   - -------------------------------------- 30.7/992.0 kB 1.3 MB/s eta 0:00:01
   ---- ----------------------------------- 112.6/992.0 kB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 276.5/992.0 kB 2.1 MB/s eta 0:00:01
   -------------------- ------------------- 512.0/992.0 kB 2.9 MB/s eta 0:00:01
   ----------------------------- ---------- 737.3/992.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------  983.0/992.0 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 992.0/992.0 kB 3.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Load the JSON data
with open('question-with-answer.json') as f:
    data = json.load(f)

# Prepare the dataset
class QADataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']
        answer = item['answer']
        input_text = f"Q: {question}\nA: {answer}"
        inputs = self.tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        return inputs

# Load tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

# Create the dataset
dataset = QADataset(data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True,
)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained("./finetuned-llama")
tokenizer.save_pretrained("./finetuned-llama")


OSError: Can't load tokenizer for 'meta-llama/Meta-Llama-3-8B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Meta-Llama-3-8B' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [34]:
!pip install transformers datasets



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
